# Import Libraries

In [1]:
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
import hashlib
from pinecone import Pinecone
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
import json
import ast
from rapidfuzz import fuzz
from datetime import datetime

c:\Codes\Python\notebooks\echo_chatbot\.chatbot\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


## Initialization

In [3]:
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

# Pinecone Initialization
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index("echo-openai")

# OpenAI Initialization
client = OpenAI(api_key=OPENAI_API_KEY)
EMBEDDINGS = OpenAIEmbeddings(model='text-embedding-3-small', openai_api_key=OPENAI_API_KEY)
LLM = ChatOpenAI(temperature=0, model_name="gpt-4-turbo", openai_api_key=OPENAI_API_KEY)

C:\Users\Zeck Lowe\AppData\Local\Temp\ipykernel_34708\480772498.py:9: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  client = OpenAI(api_key=OPENAI_API_KEY)


# Query

In [ ]:
query = "What was the QA perspective on the kickoff meeting?"

# Generate Embeddings

In [ ]:
def get_query_embeddings(query):
    """
    This function returns a list of the embeddings for a given query
    """
    query_embeddings = EMBEDDINGS.embed_query(query)
    print("Generating Embeddings: Done!")
    return query_embeddings

query_embeddings = get_query_embeddings(query=query)
print(query_embeddings)

# Metadata Filtering

## Fuzzy Match

In [ ]:
def fuzzy_match(title1, title2, threshold=80):
    """
    Perform a fuzzy match between two titles using RapidFuzz.
    Returns True if the similarity score is above the threshold.
    """
    similarity_score = fuzz.partial_ratio(title1.lower(), title2.lower())
    return similarity_score >= threshold

## Extract Metadata

In [ ]:
def extract_metadata_from_query(query):
  prompt = f"""
  You are a helpful assistant. Extract the meeting title and the meeting date from the following query.
  If the meeting title or date is not explicitly mentioned, return 'unknown'.
  If the date is mentioned as word, it should be formatted as 'YYYY-MM-DD'

  Query: {query}

  Provide the meeting title and date as a Python dictionary in this format:
  {{"meeting_title": "title_here", "date": "date_here"}}
  """

  response = LLM.invoke(prompt)
  metadata_str = response.content.strip()
  metadata_dict = ast.literal_eval(metadata_str)
  return metadata_dict

metadata = extract_metadata_from_query(query)
print(metadata)

# Query Pinecone Index

In [ ]:
def query_pinecone_index(query_embeddings, meeting_title, date, top_k=2, include_metadata=True):
    """
    Query a Pinecone index.
    """
    filter_conditions = {}
    if date.lower() != 'unknown':
      filter_conditions['date'] = date
    # if meeting_title.lower() != 'unknown':
    #   filter_conditions['title'] = meeting_title
    # if date.lower() != 'unknown':
    #   filter_conditions['date'] = date

    query_response = index.query(
        vector=query_embeddings,
        filter=filter_conditions,
        top_k=top_k,
        include_metadata=include_metadata,
        namespace="USJ-R") # Filter based on metadata
    print(query_response)

    filtered_matches = []
    for match in query_response['matches']:
      if 'metadata' in match and 'title' in match['metadata']:
        metadata_title = match['metadata']['title']
        if fuzzy_match(meeting_title, metadata_title):
          filtered_matches.append(match)

    if not filtered_matches:
      return query_response

    query_response['matches'] = filtered_matches

    print("Querying Pinecone Index: Done!")
    return query_response

answers = query_pinecone_index(query_embeddings=query_embeddings, meeting_title="kickoff meeting", date="unknown")
print(answers)

# Combining Text from Multiple Document Matches

In [ ]:
text_answer = " ".join([doc['metadata']['text'] for doc in answers['matches']])
print(text_answer)

## Prompt

In [ ]:
prompt = f"""You are a meeting facilitator.
        This user will ask you a questions about the conversation of the meeting.
        Use following piece of context to answer the question.
        If you don't know the answer, just say you don't know.
        Keep the answer within 2 sentences and concise.
        Context: {text_answers}
        Question: {query}"""

print(prompt)

# LLM

In [ ]:
def better_query_response(prompt):
    """
    This function returns a better response using LLM
    """
    better_answer = LLM.invoke(prompt)
    print("Generating Better Response: Done!")
    return better_answer

final_answer = better_query_response(prompt=prompt)
print(final_answer)

# Chatbot Response

In [ ]:
def Chatbot(query, meeting_title=None, date=None):
    print(query)
    metadata = extract_metadata_from_query(query)
    print(metadata)
    meeting_title = metadata.get('meeting_title', 'unknown')
    date = metadata.get('date', 'unknown')

    query_embeddings = get_query_embeddings(query=query)

    answers = query_pinecone_index(
        query_embeddings=query_embeddings,
        meeting_title=meeting_title,
        date=date
        )
    print(answers)

    text_answers = " ".join([doc['metadata']['text'] for doc in answers['matches']])
    print(text_answers)
    final_answer = better_query_response(prompt=prompt)
    return final_answer.content

response = Chatbot(query)
print(response)